In [1]:
from utils import read_df_parquet

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [2]:
model = MultinomialNB()

In [3]:
data = read_df_parquet('bag_of_words_df.parquet')

In [4]:
data.sample(3)

,1000km,100km,10ampliar,10km,119c10201210,11o,12e3,12milhoes,12o,1350km,...,zituo,zn,zogbi,zona,zonas,zoneamento,zr1,zum,y,corpora
2458,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Educação,seguranca escolas bibliotecas periferia vigila...
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Cultura,promover mostras cinema eventos musica shows e...
3012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Habitação,plano municipal habitacao aprovar plano munici...


In [5]:
data.shape

(3991, 15387)

In [6]:
data['y'].unique()

array(['Meio Ambiente', 'Outros temas', 'Urbanismo e Licenciamento',
       'Desenvolvimento Econômico e Trabalho',
       'Zeladoria Urbana e melhorias de bairro', 'Assistência Social',
       'Saúde', 'Habitação', 'Educação', 'Transportes e Mobilidade', None,
       'Infraestrutura Urbana e Obras', 'Cultura', 'Segurança Urbana',
       'Pessoa com Deficiência', 'Esporte e Lazer',
       'Inovação e Tecnologia', 'Direitos Humanos e Cidadania'],
      dtype=object)

In [7]:
data['y'].value_counts()

y
Saúde                                     534
Transportes e Mobilidade                  463
Zeladoria Urbana e melhorias de bairro    437
Desenvolvimento Econômico e Trabalho      359
Outros temas                              343
Assistência Social                        281
Meio Ambiente                             268
Habitação                                 226
Educação                                  203
Cultura                                   151
Segurança Urbana                          141
Direitos Humanos e Cidadania              127
Urbanismo e Licenciamento                 121
Infraestrutura Urbana e Obras             121
Esporte e Lazer                            61
Inovação e Tecnologia                      55
Pessoa com Deficiência                     31
Name: count, dtype: int64

In [8]:
data.shape

(3991, 15387)

In [9]:
data['y'].isnull().any()

np.True_

In [10]:
data['y'].isnull().mean()

np.float64(0.017288900025056377)

In [11]:
data['y'].isnull().sum()

np.int64(69)

In [12]:
data = data[data['y'].notnull()].reset_index(drop=True)

In [13]:
data['y'].value_counts()

y
Saúde                                     534
Transportes e Mobilidade                  463
Zeladoria Urbana e melhorias de bairro    437
Desenvolvimento Econômico e Trabalho      359
Outros temas                              343
Assistência Social                        281
Meio Ambiente                             268
Habitação                                 226
Educação                                  203
Cultura                                   151
Segurança Urbana                          141
Direitos Humanos e Cidadania              127
Urbanismo e Licenciamento                 121
Infraestrutura Urbana e Obras             121
Esporte e Lazer                            61
Inovação e Tecnologia                      55
Pessoa com Deficiência                     31
Name: count, dtype: int64

In [14]:
y = data['y']
X = data.drop(['y', 'corpora'], axis=1).values

In [15]:
# Definir a estratégia de validação cruzada estratificada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [16]:
scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")

In [17]:
scores

array([0.12468193, 0.13231552, 0.13010204, 0.13265306, 0.18877551,
       0.14795918, 0.11479592, 0.15816327, 0.13520408, 0.14030612])

In [18]:
print(f"Acurácia média: {scores.mean():.2f} ± {scores.std():.2f}")

Acurácia média: 0.14 ± 0.02


In [19]:
scores = cross_val_score(model, X, y, cv=cv, scoring="f1_macro")

In [20]:
print(f"f1 média: {scores.mean():.2f} ± {scores.std():.2f}")

f1 média: 0.06 ± 0.01


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
model.fit(X_train, y=y_train)

MultinomialNB()

In [24]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, log_loss

In [25]:
# Predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)  # Probabilities for log loss

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
logloss = log_loss(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)
print(f"Log Loss: {logloss:.4f}")


Accuracy: 0.1529
Classification Report:
                                         precision    recall  f1-score   support

                    Assistência Social       0.00      0.00      0.00        58
                               Cultura       0.00      0.00      0.00        36
  Desenvolvimento Econômico e Trabalho       0.14      0.13      0.13        77
          Direitos Humanos e Cidadania       0.00      0.00      0.00        28
                              Educação       0.14      0.02      0.04        45
                       Esporte e Lazer       0.00      0.00      0.00         6
                             Habitação       0.33      0.05      0.09        41
         Infraestrutura Urbana e Obras       0.00      0.00      0.00        22
                 Inovação e Tecnologia       0.00      0.00      0.00        14
                         Meio Ambiente       0.00      0.00      0.00        45
                          Outros temas       0.15      0.14      0.14        7

/home/h-pgy/projects/classifier_contrib_pdm/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h-pgy/projects/classifier_contrib_pdm/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h-pgy/projects/classifier_contrib_pdm/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_